# To Do:
#i have a terrible idea: but wait wtf i dont tuple it later idk why its not working
Figure out Jax
Figure out fun with adagrad/etc
Figure out cnn
AND THEN WHAT I DID THIS ALL FOR! GRADIENT ASCENT BABY

In [9]:
%load_ext autoreload
%autoreload 2
import jax.numpy as jnp
from jax import grad
import jax
import numpy as np
import math
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def weight_matrix(shape, naive=False):
    """weight matrix thingy.give dims. Not 0."""
    number = 1
    if(type(shape) == int):
        shape = [shape]
    for i in shape:
        number*= i
    if naive:
        return jnp.array([(i / 10) for i in range(number)]).reshape(*shape)
    return np.array([np.random.uniform(low=-.2, high=.2, size=None) for i in range(number)]).reshape(*shape)
    # return np.array([variable(np.random.uniform(low=-.2, high=.2, size=None)) for i in range(sizes[0] * sizes[1])).reshape(*shape)

In [8]:
    
def sigmoid(x):
    return jnp.vectorize(lambda x: 1/(1+math.e**-x))(x)


def relu(x):
    return jnp.where(x <= 0, 1e-2 * x, x)


In [25]:
   
class Model():
    def __init__(self, input_size, output_size, hidden_layers, naive=False, seed=None):
        '''
        Takes list of # of things in their layers.
        Layers are outputs?
        '''
        if seed != None:
            np.random.seed(seed)

        self.layer_sizes = hidden_layers
        self.layers = []
        self.biases = []
        
        prev_size = input_size
        
        for hidden_layer in hidden_layers:
            self.layers.append(weight_matrix([prev_size, hidden_layer], naive))
            self.biases.append(weight_matrix(hidden_layer, naive))
            prev_size  = hidden_layer
            
        self.biases.append(weight_matrix([output_size]))
        self.layers.append(weight_matrix([prev_size, output_size]))

        self.layers= tuple(self.layers)
        self.biases = tuple(self.biases)  
 
  
    def fd(self, x):
        '''f pass with input. '''

        for i in range(len(self.layers)):
            x = x @ self.layers[i]
            x += self.biases[i]
            if i != len(self.layers) - 1:
                x = relu(x)
            else:
                x = sigmoid(x)
            # self.hidden_states_activation.append(x)

        return x
        
    def loss_static(self, params, x, y):
        '''f pass with for loss.  '''
        w, b = params
        for i in range(len(b)):
            x = x @ w[i]
            x += b[i]
            if i != len(b) - 1:
                x = relu(x)
            else:
                x = sigmoid(x)

        y = jnp.array(y)
        return jnp.sum(x * x - 2 * x * y + y * y)

            
    def train_epoch(self, x, y, lr=10**-2):
        '''
        f pass and then uh gradient descent?

        x:  
        y: the goal. In not sparse tensor.
        lr: how quick it learns
        '''
        losses = []
        x = np.array(x)
        
        for batch_num in range(len(y)):
            mse, grads = jax.value_and_grad(self.loss_static, argnums=(0))((self.layers, self.biases), x[batch_num], y[batch_num])
            
            losses.append(mse)
            
           #0 contains weights and 1 contains the bias grads. 
            # print([i.shape for i in grads[1]])
            # print([i.shape for i in grads[0]])

            self.layers = list(self.layers)
            self.biases = list(self.biases)
            
            for i, layer in enumerate(self.layers):
                print(f"Layer {i} avg grad:", jnp.mean(grads[0][i]))


            for i, (layer, grad_layer) in enumerate(zip(self.layers, grads[0])):
                self.layers[i] = layer - lr * grad_layer  
                
            for i, (bias, grad_bias) in enumerate(zip(self.biases, grads[1])):
                self.biases[i] = bias - lr * grad_bias   
                
        preds = self.fd(x[batch_num]) 
        
        correct = jnp.sum(jnp.argmax(preds, axis=1) == jnp.argmax(y[batch_num], axis=1))
        acc = correct / len(y[batch_num])
        print(f"Acc: {acc} Loss: {mse}")
        return losses

# if this doesnt work ima cry

In [2]:
import tensorflow as tf
import keras
%load_ext autoreload
%autoreload 2

import numpy as np


In [3]:

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data(path="mnist.npz", )
indices = np.arange(len(x_train))
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

In [4]:
def batch(x, y, batch_size=32):
    if len(x) % batch_size != 0:
        x = x[:batch_size * (len(x)//batch_size)]
        y=y[:batch_size*(len(x)//batch_size)]
    print(len(x)/batch_size)
    return np.array(np.split(x, int(len(x) / batch_size), axis=0)), np.split(y, int(len(y)/batch_size), axis=0)

def fix_data(x, y):
    x = x.reshape(x.shape[0], 28*28)/255
    test = np.zeros((x.shape[0], 10))
    test[np.arange(x.shape[0]),y] = 1
    return (x, test)

In [5]:
fixed_x, fixed_y = fix_data(x_train[:10000], y_train[:10000])
b_x , b_y = batch(fixed_x, fixed_y)


312.0


In [26]:
my_model = Model(28*28, 10, [4, 8])
datas = []
for _epoch in range(10):
    datas.append(my_model.train_epoch(b_x, b_y, lr=1e-3))

Layer 0 avg grad: -0.015709033
Layer 1 avg grad: -0.17767176
Layer 2 avg grad: 0.6838916
Layer 0 avg grad: -0.0153417885
Layer 1 avg grad: -0.2015289
Layer 2 avg grad: 0.7275296
Layer 0 avg grad: -0.00813491
Layer 1 avg grad: -0.29927894
Layer 2 avg grad: 0.7144414
Layer 0 avg grad: -0.013045574
Layer 1 avg grad: -0.2535523
Layer 2 avg grad: 0.71853024
Layer 0 avg grad: -0.0035471732
Layer 1 avg grad: -0.18868366
Layer 2 avg grad: 0.6662707
Layer 0 avg grad: -0.0070924186
Layer 1 avg grad: -0.22669001
Layer 2 avg grad: 0.7357151
Layer 0 avg grad: -0.016863491
Layer 1 avg grad: -0.25603843
Layer 2 avg grad: 0.7303526
Layer 0 avg grad: -0.012690411
Layer 1 avg grad: -0.17700016
Layer 2 avg grad: 0.73039114
Layer 0 avg grad: -0.014897702
Layer 1 avg grad: -0.2825428
Layer 2 avg grad: 0.7259692
Layer 0 avg grad: -0.0103717465
Layer 1 avg grad: -0.17774928
Layer 2 avg grad: 0.7133192
Layer 0 avg grad: -0.01079789
Layer 1 avg grad: -0.22263922
Layer 2 avg grad: 0.7306553
Layer 0 avg grad: -0

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

fig = plt.figure(figsize=(10, 7))
pic = 1
for i, img in enumerate(x_test[:10]):
  plt.subplot(2, 5, pic)
  plt.axis('off')
  predicted = my_model.fd(img.flat)
  keras_pred = model.predict(img.reshape(1, 28*28))
  plt.title(f"T {y_test[i]} mine {np.argmax(predicted)} keras {np.argmax(keras_pred)} ")
  plt.imshow(img)
  pic+= 1
plt.show()
#60% acc. Considering this is from nearly scratch not terrible 